In [1]:
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Union, Optional
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Data Cleaning

In [3]:
def drop_nan_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Drop rows containing NaN values from the specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame to process.
        columns (List[str]): A list of column names to check for NaN values.

    Returns:
        pd.DataFrame: The updated DataFrame with NaN-containing rows dropped.

    Raises:
        ValueError: If any of the specified columns are not present in the DataFrame.
    """
    # Validate that all specified columns exist in the DataFrame
    missing_columns = set(columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Columns not found in DataFrame: {', '.join(missing_columns)}")

    # Identify columns with NaN values
    nan_columns = df[columns].columns[df[columns].isna().any()].tolist()

    # Log dropped rows if any
    if nan_columns:
        nan_count = df[columns].isna().sum()
        logger.info("Dropping rows with NaN values:")
        for col in nan_columns:
            logger.info(f"  {col}: {nan_count[col]} rows")

    # Drop rows with NaN values in specified columns
    original_row_count = len(df)
    df_cleaned = df.dropna(subset=columns)
    dropped_row_count = original_row_count - len(df_cleaned)

    if dropped_row_count > 0:
        logger.info(f"Total rows dropped: {dropped_row_count}")
    else:
        logger.info("No rows were dropped.")

    return df_cleaned

In [4]:
def remove_duplicates(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicates from a DataFrame.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame with duplicates removed.
    """
    num_duplicates = data.duplicated().sum()
    
    if num_duplicates > 0:
        logging.info(f"{num_duplicates} duplicates found.")
        data = data.drop_duplicates(keep="first").reset_index(drop=True)
        logging.info(f"{num_duplicates} duplicates removed.")
    else:
        logging.info("No duplicates found.")
    
    return data

In [5]:
def create_date_column(df: pd.DataFrame, column_names: List[str], new_col: str) -> pd.DataFrame:
    """
    Creates a new date column in the DataFrame by concatenating the values of three specified columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_names (List[str]): A list of three column names to be concatenated [year, month, day].
        new_col (str): The name of the new date column to be created.

    Returns:
        pd.DataFrame: The DataFrame with the new date column added.

    Raises:
        ValueError: If the input list doesn't contain exactly three column names or if columns are missing.
    """
    if len(column_names) != 3:
        raise ValueError("column_names must contain exactly three elements: [year, month, day]")

    year_col, month_col, day_col = column_names

    # Check if all required columns exist in the DataFrame
    missing_columns = set(column_names) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {', '.join(missing_columns)}")

    # Create copies to avoid SettingWithCopyWarning
    df = df.copy()

    try:
        # Convert year and day columns to integers
        df[year_col] = df[year_col].astype(float).astype(int)
        df[day_col] = df[day_col].astype(float).astype(int)

        # Concatenate the columns to create a date string
        df[new_col] = (df[year_col].astype(str) + '-' + 
                       df[month_col].astype(str) + '-' + 
                       df[day_col].astype(str))

        # Convert to datetime
        df[new_col] = pd.to_datetime(df[new_col], errors='coerce')

        # Log information about the conversion
        valid_dates = df[new_col].notna().sum()
        logger.info(f"Created new date column '{new_col}'. Valid dates: {valid_dates}/{len(df)}")

    except Exception as e:
        logger.error(f"Error creating date column: {str(e)}")
        raise

    return df

In [6]:
def apply_title_case(text):
    """
    Apply title case to a given string.
    
    Args:
        text: The input string to process.
    
    Returns:
        str: The processed string in title case.
    """
    if pd.isna(text):
        return np.nan
    if not isinstance(text, str):
        logger.warning(f"Non-string value encountered: {text}")
        return str(text)
    return text.title()

In [7]:
def process_outcome_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the 'outcome' column of the DataFrame by applying title case.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing the 'outcome' column.
    
    Returns:
        pd.DataFrame: The DataFrame with the processed 'outcome' column.
    """
    if 'outcome' not in df.columns:
        logger.error("'outcome' column not found in the DataFrame")
        return df

    original_null_count = df['outcome'].isnull().sum()
    
    df['outcome'] = df['outcome'].apply(apply_title_case)
    
    new_null_count = df['outcome'].isnull().sum()
    if new_null_count > original_null_count:
        logger.warning(f"Number of null values in 'outcome' increased from {original_null_count} to {new_null_count}")
    
    non_string_count = df['outcome'].apply(lambda x: not isinstance(x, str) if pd.notna(x) else False).sum()
    if non_string_count > 0:
        logger.warning(f"Found {non_string_count} non-string values in 'outcome' after processing")

    return df

In [8]:
def drop_null_values(df: pd.DataFrame, column_name: str = 'outcome') -> pd.DataFrame:
    """
    Drop rows from the DataFrame where the specified column contains null values.

    Args:
        df (pd.DataFrame): The DataFrame from which to drop rows.
        column_name (str): The name of the column to check for null values. Default is 'outcome'.

    Returns:
        pd.DataFrame: The DataFrame with rows containing null values in the specified column dropped.
    """
    initial_row_count: int = df.shape[0]
    cleaned_df: pd.DataFrame = df.dropna(subset=[column_name])
    final_row_count: int = cleaned_df.shape[0]
    dropped_row_count: int = initial_row_count - final_row_count
    
    logging.info(f"Total dropped rows with null values in '{column_name}': {dropped_row_count}")
    
    return cleaned_df

In [9]:
def categorize_case(case_type: str, criminal_cases: Optional[List[str]]) -> str:
    """
    Categorize a case as 'Criminal' or 'Civil' based on its type.
    
    Args:
        case_type (str): The type of the case.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
        
    Returns:
        str: 'Criminal' if the case type is in the criminal cases list or if criminal_cases is None, 'Civil' otherwise.
    """
    if criminal_cases is None:
        return 'Criminal'
    else:
        return 'Criminal' if case_type in criminal_cases else 'Civil'

In [10]:

def categorize_cases(df: pd.DataFrame, criminal_cases: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Categorize all cases in the DataFrame as 'Criminal' or 'Civil'.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing case data.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
            If None, all cases are categorized as 'Criminal'.
        
    Returns:
        pd.DataFrame: DataFrame with an added 'nature' column indicating case nature.
    """
    df['nature'] = df['case_type'].apply(lambda x: categorize_case(x, criminal_cases))

    # Check for presence of both case types
    if 'Criminal' not in df['nature'].values:
        logging.warning("No criminal cases found in the DataFrame.")
    if 'Civil' not in df['nature'].values:
        logging.warning("No civil cases found in the DataFrame.")
    
    return df

In [11]:
def apply_dict(value: Any, dictionary: Dict[str, Union[str, List[Any]]]) -> Union[str, None]:
    """
    Find all keys in a dictionary where the given value matches.

    Args:
        value: The value to search for.
        dictionary: The dictionary to search in.

    Returns:
        A list of keys where the value matches, or None if no matches are found.
    """
    matching_keys = []
    for key, dict_value in dictionary.items():
        if isinstance(dict_value, str) and dict_value == value:
            matching_keys.append(key)
        elif isinstance(dict_value, list) and value in dict_value:
            matching_keys.append(key)
    
    if not matching_keys:
        return None
    elif len(matching_keys) == 1:
        return matching_keys[0]
    else:
        return matching_keys

In [12]:
def generate_case_num(df: pd.DataFrame, court_col: str, caseid_type_col: str, caseid_no_col: str, filed_yyyy_col: str, new_col='case_number') -> pd.DataFrame:
    """
    Generates a case number by concatenating court, caseid_type, caseid_no, and filed_yyyy columns.

    Args:
        df (pd.DataFrame): The dataframe containing the necessary columns.
        court_col (str): The name of the column containing court information.
        caseid_type_col (str): The name of the column containing case ID type.
        caseid_no_col (str): The name of the column containing case ID number.
        filed_yyyy_col (str): The name of the column containing the year the case was filed.
        new_col (str): The name of the new column to be created for the case number. Default is 'case_num'.

    Returns:
        pd.DataFrame: DataFrame with the new case number column.
    """
    df[new_col] = df[court_col] + '/' + df[caseid_type_col] + '/' + df[caseid_no_col] + '/' + df[filed_yyyy_col].astype(str)
    return df

In [13]:

def is_concluded(outcome: str, resolved_outcomes: List[str]) -> int:
    """
    Determine if the case is concluded based on the outcome.
    
    Args:
        outcome (str): The outcome of the case.
        resolved_outcomes (List[str]): List of outcomes considered as resolved.
        
    Returns:
        int: 1 if the case outcome is resolved, 0 otherwise.
    """
    return 1 if outcome in resolved_outcomes else 0

In [14]:
def is_case_registered(outcome: str, activity_date: Union[pd.Timestamp, str], filed_date: Union[pd.Timestamp, str]) -> bool:
    """
    Determine if a case is registered based on its outcome and dates.

    Args:
        outcome (str): The outcome of the case.
        activity_date (Union[pd.Timestamp, str]): The date of the activity.
        filed_date (Union[pd.Timestamp, str]): The date the case was filed.

    Returns:
        bool: True if the case is registered, False otherwise.
    """
    try:
        # Normalize the outcome string
        normalized_outcome = outcome.strip().lower()

        # Check if the outcome indicates registration
        is_registered_outcome = 'case registered/filed' in normalized_outcome 

        # Convert dates to pd.Timestamp if they're strings
        if isinstance(activity_date, str):
            activity_date = pd.to_datetime(activity_date, errors='coerce')
        if isinstance(filed_date, str):
            filed_date = pd.to_datetime(filed_date, errors='coerce')

        # Check if dates are equal
        dates_match = pd.notna(activity_date) and pd.notna(filed_date) and activity_date == filed_date

        is_registered = is_registered_outcome and dates_match

        if is_registered:
            logger.debug(f"Case registered: outcome='{outcome}', activity_date={activity_date}, filed_date={filed_date}")
        
        return is_registered

    except Exception as e:
        logger.error(f"Error in is_case_registered: {e}")
        return False


In [15]:
def process_case_status(df: pd.DataFrame, resolved_outcome: List[str]) -> pd.DataFrame:
    """
    Process the DataFrame to add 'concluded' and 'registered' columns.

    Args:
        df (pd.DataFrame): The input DataFrame containing case information.

    Returns:
        pd.DataFrame: The DataFrame with added 'concluded' and 'registered' columns.

    Raises:
        ValueError: If required columns are missing from the DataFrame.
    """
    required_columns = ['outcome', 'activity_date', 'filed_date']
    missing_columns = set(required_columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

    #df['concluded'] = df['outcome'].apply(is_case_concluded)
    df['concluded'] = df['outcome'].apply(lambda x: is_concluded(x, resolved_outcome))
    df['registered'] = df.apply(lambda row: is_case_registered(row['outcome'], row['activity_date'], row['filed_date']), axis=1)

    logger.info(f"Processed {len(df)} cases")
    logger.info(f"Concluded cases: {df['concluded'].sum()}")
    logger.info(f"Registered cases: {df['registered'].sum()}")

    return df

In [16]:
def analyze_court_outcomes(df: pd.DataFrame, start_date: str, end_date: str, outcome: str) -> pd.DataFrame:
    """
    Calculate the number of case outcomes per court within a specified period.
    
    Args:
        df (pd.DataFrame): A pandas DataFrame containing the data.
        start_date (str): The starting date of the period (YYYY-MM-DD format).
        end_date (str): The ending date of the period (YYYY-MM-DD format).
        outcome (str): A column representing the outcome of interest.
        
    Returns:
        pd.DataFrame: A DataFrame showing the number of resolved cases per court and case category.
    """
    try:
        period_start = pd.to_datetime(start_date)
        period_end = pd.to_datetime(end_date)
        
        if period_start > period_end:
            raise ValueError("start_date must be earlier than end_date")
        
        required_columns = {'court', 'broad_case_type', 'activity_date', outcome}
        if not required_columns.issubset(df.columns):
            missing_columns = required_columns - set(df.columns)
            raise KeyError(f"Missing required columns: {missing_columns}")
        
        filtered_cases = df[
            (df['activity_date'] >= period_start) &
            (df['activity_date'] <= period_end) &
            (df[outcome] == 1)
        ]
        
        if filtered_cases.empty:
            logging.warning("No cases found for the given date range and outcome.")
   
        outcome_by_type = (
            filtered_cases
            .groupby(['court', 'broad_case_type'])
            .size()
            .reset_index(name='num_cases')
        )

        result = outcome_by_type.pivot_table(
            index='court', 
            columns='broad_case_type', 
            values='num_cases', 
            fill_value=0
        )
        
        logging.info("Successfully calculated case outcomes per court.")
        return result
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [17]:
def check_time_limit(age, case_category, concluded, time_lines):
    """
    Check if a case falls within the specified time limit for its category and is concluded.
    
    Parameters:
        age (int): The age of the case in days.
        case_category (str): The category of the case.
        concluded (int): The status of the case conclusion (1 for concluded, 0 otherwise).
        time_lines (dict): A dictionary with case categories as keys and time limits as values.
        
    Returns:
        int: 1 if the case is within the time limit and concluded, otherwise 0.
    """
    time_limit = time_lines.get(case_category, 0)
    return 1 if age <= time_limit and concluded == 1 else 0


In [18]:
def monthly_case_stats(df, registered_col, concluded_col):
    """Calculates monthly statistics for registered and concluded cases.

    Args:
        df (pandas.DataFrame): The input DataFrame containing case data.
        registered_col (str): The name of the column containing registered cases.
        concluded_col (str): The name of the column containing concluded cases.

    Returns:
        pandas.DataFrame: A DataFrame with monthly statistics for registered and concluded cases.
    """

    monthly_cases = df.groupby(['court', 'date_mon', 'case_type']).agg(
        registered=(registered_col, 'sum'),
        concluded=(concluded_col, 'sum')
    ).reset_index()

    return monthly_cases

In [19]:
def add_timely_pmmu_column(data_f, time_lines):
    data_f['timely'] = data_f.apply(lambda row: True if row['broad_case_type'] in time_lines and row['concluded'] == 1 and row['age'] <= time_lines[row['broad_case_type']] else False, axis=1)
    return data_f

In [20]:
def add_adjournment_columns(df, non_adjourned):
    """Adds 'adjourned' and 'adjournable' columns to the DataFrame."""
    df['adjourned'] = (df['reason_adj'].notnull() & df['comingfor'].apply(lambda x: x not in non_adjourned)).astype(int)
    df['adjournable'] = df['comingfor'].apply(lambda x: x not in non_adjourned).astype(int)
    return df

In [21]:
def calculate_adjourned_per_court(df):
    """Calculates adjourned cases per court and reason."""
    return df.groupby(['court', 'reason_adj'])['adjourned'].sum().reset_index(name='count')



In [22]:
def calculate_adjournment_proportion(df):
    """Calculates total adjourned, adjournable cases, and adjournment proportion per court."""
    court_summary = df.groupby('court').agg(
        total_adjourned=('adjourned', 'sum'),
        total_adjournable=('adjournable', 'sum')
    ).reset_index()
    court_summary['adjourn_proportion'] = court_summary['total_adjourned'] / court_summary['total_adjournable']
    return court_summary

In [23]:
NON_ADJOUNABLE = [
    'Taxation and Issuance of Certificates',
    'Orders',
    'Appointments of  Mediator',
    'Screening of files for Mediation',
    'Post-judgment',
    'Re-activation',
    'Reactivation',
    'Notice of Taxation',
    'Entering Interlocutory Judgments',
    'Approval by DR', 
    'Registration/Filing-Application', 
    'Registration/Filing', 
    'Registration/Filing-Application',
 ]
RESOLVED_OUTCOMES = [
    'Ruling Delivered- Case Closed', 'Grant Confirmed', 'Matter Withdrawn',
    'Dismissed For Want Of Prosecution - Case Closed', 'Dismissed',
    'Terminated/ Struck Out/ Dismissed/case Closed', 'Judgment Delivered- Case Closed',
    'Application Allowed - Case Closed',
    'Matter Settled- Case Closed', 'Consent Recorded - Case Closed',
    'Judgment Delivered', 'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted', 'Application Withdrawn - Case Closed',
    'Struck Out', 'Application Dismissed - Case Closed',
    'Out Of Court Settlement Reached', 'Terminated',
    'Ruling Delivered- Application Closed', 'Consolidated- Case Closed',
    'Interlocutory Judgement Entered', 'Abated', 'Limited Grant Issued',
    'Grant Revoked', 'Placed In Probation', 'Ruling Delivered- Accused Discharged',
    'Revision Declined', 'Retrial', 'Probation Orders Issued',
    'Matter Settled Through Mediation', 'Appeal Dismissed', 'Appeal Rejected', 
    'Order Issued - Case Closed',
    'Terminated'
]
CRIMINAL_CASES = [
    'Murder Case',
    'Criminal Revision',
    'Criminal Appeal',
    'Murder - Gender Justice Criminal Case',
    'Criminal Court Martial Appeal',
    'Anti-Corruption and Economic Crimes Revision',
    'Criminal Miscellaneous Application',
    'Criminal Applications', 
    'COA Criminal Appeal'
]
BROAD_CASE_TYPES = {
    'Civil Matter': [
        'Civil Suit',
        'Anti-Corruption and Economic Crimes Suit',
        'Family Originating Summons',
        'Family Civil Case',
        'HCC(OS) Family',
        'Commercial Admiralty',
        'Commercial Matters',
    ],
    
    'Adoption': 'Family Adoption',
    'Divorce': 'Family Divorce Cause',

    'Criminal Application': 'Criminal Miscellaneous Application',

    'Civil Application': [
        'Civil Case Miscellaneous',
        'Judicial Review Miscellaneous',
        'JR  Petition Miscellaneous',
        'Anti-Corruption and Economic Crimes Miscellaneous',
        'Commercial Miscellaneous',
        'Constitution and Human Rights Petitions Miscellaneous',
        'Family Miscellaneous',
        'Commercial Arbitration',
    ],
    'Judicial Review': [
        'Anti-Corruption and Economic Crime Judicial review',
        'Judicial Review ELC',
        'Judicial Review',
    ],
    'Criminal Revision': [
        'Criminal Revision',
        'Anti-Corruption and Economic Crimes Revision',
    ],
    'Criminal Appeal': [
        'Criminal Appeal',
        'Criminal Court Martial Appeal',
        'Anti-Corruption and Economic Crimes Appeal',
    ],
    'Civil Appeal': [
        'Family Appeal',
        'Civil Appeal',
        'Commercial Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Constitution and Human Rights Petition Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Gender Justice Civil Appeal',
        'Constitution and Human Rights Miscellaneous Election Petition Appeal (MEPA)',
    ],
    'Constitution Petition': [
        'Anti Corruption and Economic Crimes Petition',
        'High Court Criminal Petition',
        'Constitution and Human Rights Petition (Civil)',
        'Constitution and Human Rights Election Petition',
        'High Court Constitution and Human Rights Petitions (Criminal)',
        'Commercial Petition',
    ],
    'Probate Administration': [
        'Family P&A Intestate',
        'Family P&A Ad Litem',
        'Family P&A Ad Colligenda',
        'Family P&A Citation',
        'Family P&A Testate',
        'Family P&A Resealing of Grant',
        'Family P&A De Bonis Non',
        'Resealing of Grant',
        'Citation-Family',
    ],
    'Murder': [
        'Murder Case',
        'Murder - Gender Justice Criminal Case',
    ],
    'Tax Appeal': [
        'Commercial Income Tax Appeal',
        'Commercial Custom Tax Appeal',
    ],
    'Bankruptcy and Insolvency' : [
        'Commercial Insolvency Notice Petition',
        'Commercial Insolvency Petition',
        'Commercial Bankruptcy Notice',
        'Commercial Insolvency Cause',
        'Commercial Insolvency Notice',
        'Commercial Bankruptcy Cause',
        'Commercial Winding Up Cause',
    ]
}

MERIT_CATEGORY ={
    'Merit Resolution' : [    
    'Ruling Delivered- Case Closed', 
    'Judgment Delivered- Case Closed',
    'Judgment Delivered', 
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Ruling Delivered- Application Closed',
    'Retrial',
    'Ruling Delivered- Accused Discharged',
    'Appeal Dismissed',
    'Grant Revoked',
],
'Non Merit Resolution': [
    'Grant Confirmed', 
    'Matter Withdrawn',
    'Dismissed For Want Of Prosecution - Case Closed',
    'Dismissed',
    'Terminated/ Struck Out/ Dismissed/case Closed', 
    'Application Allowed - Case Closed',
    'Matter Settled- Case Closed', 
    'Consent Recorded - Case Closed',
    'Application Withdrawn - Case Closed',
    'Struck Out', 
    'Application Dismissed - Case Closed',
    'Out Of Court Settlement Reached', 
    'Terminated',
    'Consolidated- Case Closed',
    'Interlocutory Judgement Entered', 
    'Abated', 
    'Limited Grant Issued',
    'Placed In Probation', 
    'Revision Declined',  
    'Probation Orders Issued',
    'Matter Settled Through Mediation', 
    'Appeal Rejected', 
    'Order Issued - Case Closed',
    'Terminated'  
    ]
} 


productivity = {
    'Judgment Delivered': [
        'Judgment Delivered- Case Closed',
        'Judgment Delivered',
        'Judgment Delivered- Acquittal',
        'Judgment Delivered- Convicted',
        'Grant Revoked',
        'Retrial'
        ],
    'Ruling Case Closed': [
        'Ruling Delivered- Case Closed', 
        'Ruling Delivered- Accused Discharged',
        ],
    'Final Grant': [
        'Grant Confirmed',
        'Limited Grant Issued',
        ],
    'Case Withdrawn': [
        'Matter Withdrawn',
        'Application Withdrawn - Case Closed',
        ],
   'Out Of Court Settlement': [
        'Consent Recorded - Case Closed',
        'Matter Settled Through Mediation',
        'Out Of Court Settlement Reached',
    ],
    'Dismissed':[
        'Dismissed For Want Of Prosecution - Case Closed',
        'Dismissed',
        'Appeal Dismissed',
        'Terminated',
        'Terminated/ Struck Out/ Dismissed/case Closed',
    ],
    'Case Transfered': [
        'File Transfered -case Closed',
        'File Transferred',
    ],
    'Case Closed': [
        'Struck Out',
        'Application Dismissed - Case Closed',
        'Application Allowed - Case Closed',
        'Matter Settled- Case Closed',
        'Ruling Delivered- Application Closed',
        'Consolidated- Case Closed',
        'Abated',
        'Placed In Probation',
        'Revision Declined',
        'Probation Orders Issued',
        'Appeal Rejected',
        'Interlocutory Judgement Entered',
    ],
}
PMMU_TIME_LINES = {
    'murder': 360,
    'revision': 90,
    'misc_application': 90,
    'suit': 360,
    'judicial_review': 180,
    'constitutional_petition': 180,
}
HC_PMMU_TIME_LINES = {
    'Murder': 365, 
    'Constitution Petition': 180, 
    'Criminal Revision': 90, 
    'Judicial Review': 180, 
    'Civil Matter': 365
}
# transferred
TRANSFERED_CASES = ['File Transfered -case Closed', 
        'File Transferred',]

In [24]:
file_path = '/home/stanoo/Projects/data/hc' 
raw_df = pd.read_csv(f'{file_path}/q2-2024-25-hc.csv')

/tmp/ipykernel_108622/1332518533.py:2: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(f'{file_path}/q2-2024-25-hc.csv')


In [25]:
df = raw_df.copy()

In [26]:
df['outcome'] = df['outcome'].replace('Terminated/ Struck Out/ Dismissed/Case Closed', 'Terminated')

In [27]:
df.columns

Index(['court', 'date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'original_court',
       'original_code', 'original_number', 'original_year', 'case_type',
       'judge_1', 'judge_2', 'judge_3', 'judge_4', 'judge_5', 'judge_6',
       'judge_7', 'comingfor', 'outcome', 'reason_adj', 'next_dd', 'next_mon',
       'next_yyyy', 'male_applicant', 'female_applicant',
       'organization_applicant', 'male_defendant', 'female_defendant',
       'organization_defendant', 'legalrep', 'applicant_witness',
       'defendant_witness', 'custody', 'other_details'],
      dtype='object')

In [61]:

# df = df[[df.columns[-1]] + list(df.columns[:-1])]

In [29]:
df = create_date_column(df.copy(), ['date_dd', 'date_mon', 'date_yyyy'], 'activity_date')
df = create_date_column(df.copy(), ['filed_dd', 'filed_mon', 'filed_yyyy'], 'filed_date')
df = drop_nan_columns(df, ['date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'case_type', 'comingfor'])
df = remove_duplicates(df)
df = df.apply(lambda x: x.strip().title() if isinstance(x, str) else x)
df = process_outcome_column(df)
df = drop_null_values(df)
df = categorize_cases(df, CRIMINAL_CASES)
df['broad_case_type'] = df['case_type'].apply(lambda x: apply_dict(x, BROAD_CASE_TYPES))
df = generate_case_num(df, 'court', 'caseid_type', 'caseid_no', 'filed_yyyy')
df = process_case_status(df, RESOLVED_OUTCOMES)
df['merit_category'] = df['outcome'].apply(lambda x: apply_dict(x, MERIT_CATEGORY))
df['age'] = (df['activity_date'] - df['filed_date']).dt.days
df = add_timely_pmmu_column(df, HC_PMMU_TIME_LINES)
df['comingfor'] = df['comingfor'].str.strip()
df = add_adjournment_columns(df, NON_ADJOUNABLE)
df['transfer'] = df['outcome'].apply(lambda x: x in TRANSFERED_CASES)

2025-01-07 13:08:11,719 - INFO - Created new date column 'activity_date'. Valid dates: 238128/238128
2025-01-07 13:08:12,201 - INFO - Created new date column 'filed_date'. Valid dates: 238128/238128
2025-01-07 13:08:12,293 - INFO - Dropping rows with NaN values:
2025-01-07 13:08:12,293 - INFO -   caseid_no: 8 rows
2025-01-07 13:08:12,363 - INFO - Total rows dropped: 8
2025-01-07 13:08:12,528 - INFO - 1532 duplicates found.
2025-01-07 13:08:12,748 - INFO - 1532 duplicates removed.
2025-01-07 13:08:13,012 - INFO - Total dropped rows with null values in 'outcome': 2920
2025-01-07 13:08:15,001 - INFO - Processed 233668 cases
2025-01-07 13:08:15,002 - INFO - Concluded cases: 22760
2025-01-07 13:08:15,003 - INFO - Registered cases: 25084


In [ ]:
filed_cases = analyze_court_outcomes(df, '2024-07-01', '2024-12-31', 'registered')
resolved_cases = analyze_court_outcomes(df, '2024-07-01', '2024-12-31', 'concluded')
monthly_filed_cases = df.groupby(['court','date_mon']).agg({'registered':'sum'}).reset_index()
monthly_concluded_cases = df.groupby(['court','date_mon']).agg({'concluded':'sum'}).reset_index()
average_time_to_conclude = df.loc[df['concluded'] == 1].pivot_table(index='court', columns='nature', values='age', aggfunc='mean', fill_value=0).round(2)
monthly_stats = monthly_case_stats(df, 'registered', 'concluded')
court_productivity = df.pivot_table(index='court', columns='merit_category', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None)
total_resolved_cases = df.pivot_table( index='court', columns='broad_case_type', values='timely', aggfunc=lambda x: np.sum(df.loc[x.index, 'concluded'] == 1), fill_value=0)
timely_resolved_cases = df[df['timely']].pivot_table(index='court', columns='broad_case_type', values='timely', aggfunc='sum', fill_value=0)
percentage_timely = (timely_resolved_cases / resolved_cases ).fillna(0)
adjourned_per_court = calculate_adjourned_per_court(df)
adjourned_proportion = calculate_adjournment_proportion(df)
transfered_cases = df[df['transfer']].pivot_table(index='court', columns='broad_case_type', values='transfer', aggfunc='sum', fill_value=0)

2025-01-07 13:08:48,899 - INFO - Successfully calculated case outcomes per court.
2025-01-07 13:08:48,917 - INFO - Successfully calculated case outcomes per court.


In [34]:
df[(df['court'] == 'Nyeri') & (df['merit_category'] == 'Merit Resolution')].groupby('judge_1').size().reset_index(name='count').sort_values('count', ascending=False)

,judge_1,count
8,"Magare, Dennis Kizito",136
4,"Angima, Edina Nyaboke",85
13,"Odero, Maureen Akinyi",47
7,"Kendagor, Caroline J",10
9,"Mbungi, Stephen",10
12,"Njoroge, Benjamin Kimani",10
2,"Adagi, Noel Inziani Onditi",9
14,"Rutto, Rhoda",9
11,"Muteti, Alexander Muasya",9
10,"Micheni, Wendy K.",8


In [35]:
output_path = '/home/stanoo/Projects/data/hc/reports'

In [37]:
court_productivity = df.groupby(['judge_1', 'court'])['concluded'].sum().reset_index(name='count')
matters_handled = df.groupby(['court','judge_1'])['court'].count().reset_index(name='count')
#judge_df = df[df['judge_1'].isin(judge_names['judge_1'])]
#judge_productivity = judge_df.pivot_table(index='judge_1', columns='productivity', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None)

In [38]:
column_order = ['Murder', 'Criminal Appeal', 'Criminal Application', 
                'Criminal Revision', 'Civil Matter', 'Civil Appeal', 'Civil Application', 
                'Constitution Petition', 'Judicial Review',
                  'Bankruptcy and Insolvency', 'Tax Appeal', 
                  'Adoption', 'Divorce', 'Probate Administration']

resolved_cases.to_csv(f'{output_path}/resolved_cases.csv,', columns=column_order, index=True)
#monthly_filed_cases.to_csv(f'{output_path}/filed_per_month.csv', index=False)
#monthly_concluded_cases.to_csv(f'{output_path}/monthly_concluded.csv', index=False)
court_productivity.to_csv(f'{output_path}/court_productivity.csv')
average_time_to_conclude.to_csv(f'{output_path}/average_time_to_conclude.csv')
percentage_timely.to_csv(f'{output_path}/resolved_within_timeline.csv')
adjourned_per_court.to_csv(f'{output_path}/adjournement.csv', index=False)
adjourned_proportion.to_csv(f'{output_path}/adjourned_proportion.csv', index=False)

In [39]:
column_order = ['Murder', 'Criminal Appeal', 'Criminal Application', 
                'Criminal Revision', 'Civil Matter', 'Civil Appeal', 'Civil Application', 
                'Constitution Petition', 'Judicial Review',
                  'Bankruptcy and Insolvency', 
                  'Adoption', 'Probate Administration']
transfered_cases.to_csv(f'{output_path}/transfered_cases.csv', columns=column_order, index=True) 
filed_cases.to_csv(f'{output_path}/filed_cases.csv', columns=column_order, index=True)


### Determine Productivity per judge (Merit/Non Merit)

In [129]:
baseline = pending_baseline.groupby('court')['case_number'].count().reset_index(name='total_pending')
#baseline.to_csv(f'{output_path}/hc_baseline.csv', index=False)
total_registered = df.groupby('court')['registered'].sum().reset_index(name='total_filed')
# trim & strip the court names
baseline['court'] = baseline['court'].str.strip()
# merge total_registered with baseline on 'court' without skipping missing
baseline_total_registered = pd.merge(baseline, total_registered, on='court', how='outer')

In [ ]:
pending_baseline.groupby('court')['case_number'].count().sort_values(ascending=False).reset_index(name='total_cases')
pending_baseline['case_number'] = pending_baseline ['court'] + "/" + pending_baseline['case_number']
pending_baseline['case_number'] = pending_baseline['case_number'].str.replace(' ', '')
pending_df = df.merge(pending_baseline, on=['case_number', 'case_type', 'court'], how='left')
pending_df['unique_number'] = pending_df['court'] + '/' + pending_df['caseid_type'] + '/' + pending_df['caseid_no'] + '/' + pending_df['filed_yyyy'].astype(str)
pending_df = pending_df.sort_values(by=['court', 'activity_date', 'unique_number'], ascending=True)

In [188]:
def set_concluded_and_resolved(df):
    # Set 'resolved' column based on the 'concluded' column within each group of 'number_on_file'
    df['resolved'] = df.groupby('unique_number')['concluded'].transform('max')
    return df
def preprocess_dataframe(df):
    # Sort DataFrame by 'activity_date' in descending order
    df_sorted = df.sort_values(by='activity_date', ascending=False)
    
    # Drop duplicates based on 'number_on_file' while keeping the first occurrence
    df_unique_cases = df_sorted.drop_duplicates(subset='unique_number', keep='first')
    
    # Filter unresolved cases
    unique_unresolved_cases = df_unique_cases[df_unique_cases['resolved'] == 0]
    

    # Reset index
    unique_unresolved_cases = unique_unresolved_cases.reset_index(drop=True)
    
    return unique_unresolved_cases

In [ ]:
final_df = set_concluded_and_resolved(pending_df)
#final_df.groupby('resolved')['resolved'].value_counts().sort_values(ascending=False)
processed_df = preprocess_dataframe(final_df)
#processed_df.groupby('court')['court'].value_counts().sort_values(ascending=False)
pending_cases = processed_df.groupby('court')['court'].value_counts().sort_values(ascending=False)

In [ ]:
pending_baseline['case_number'] = pending_baseline ['court'] + "/" + pending_baseline['case_number']

In [ ]:
df = ['court', 'comingfor', 'outcome', 'activity_date', 'filed_date', 'activity_date_year', 'activity_date_month', 'nature', 'case_category', 'case_number']

In [ ]:
# Convert both dataframes to sets of 'case_number' values
pending_baseline_cases = set(pending_baseline['case_number'].unique())

df_cases = set(df['case_number'].unique())

# Find the cases present in df but not in df_a
cases_not_in_pending = df_cases.difference(pending_baseline_cases)

# Print the cases
print("Cases present in df but not in df_a:")
cases_not_in_pending

In [138]:
cases_not_in_pending = pd.DataFrame(cases_not_in_pending)

In [141]:
cases_not_in_pending.shape

(84810, 1)

In [132]:
combined_df = pd.merge(df, pending_baseline, on='case_number', how='left', indicator=True)

In [133]:
combined_df = combined_df[combined_df['_merge'] == 'left_only'].drop(columns='_merge')

In [ ]:
combined_df

In [ ]:
combined_df = combined_df.sort_values(by=['court', 'activity_date', 'case_number'], ascending=True)

In [ ]:
combined_df

In [142]:
# Convert df_a['case_number'] to a set
df_a_cases = set(pending_baseline['case_number'].unique())


In [146]:
filtered_df_closed = df[(df['concluded'] == 1) & (df['case_number'].isin(df_a_cases))]

In [ ]:
filtered_df_closed

In [143]:
# Filter df based on 'outcome' == 1 and 'case_number' not in df_a_cases
filtered_df = df[(df['concluded'] == 1) & ~(df['case_number'].isin(df_a_cases))]

In [ ]:
filtered_df.groupby('court')['case_number'].count().sort_values(ascending=False).reset_index(name='count')

In [ ]:
filtered_df['resolved'] = filtered_df.groupby('case_number')['concluded'].transform('max')

In [152]:
  # Drop duplicates based on 'number_on_file' while keeping the first occurrence
filtered_df = filtered_df.drop_duplicates(subset='case_number', keep='first')
    

In [156]:
nakuru = filtered_df[filtered_df['court'] == 'Milimani Anti Corruption and Economic Crimes']

In [157]:
nakuru.to_csv(f'{output_path}/nakuru.csv', index=False)

In [158]:
pending_baseline.to_csv(f'{output_path}/pending_baseline_2.csv', index=False)

In [ ]:
filtered_df.groupby('court')['case_number'].count().sort_values(ascending=False).reset_index(name='count')

In [145]:
pending_baseline[pending_baseline['case_number'] == 'Busia/HCCCMISC/E026/2023']

,court,case_number,case_type


#### Backlog determination

In [ ]:
#import pandas as pd
# raw data of pending cases(next period baseline)
#df = pd.read_csv(f'{output_path}/raw_pending_cases.csv')

In [ ]:
# The date for computation of backlog ought to be the end of the quarter
quarter_end = pd.to_datetime('2024-03-31')
df['end_date'] = quarter_end

In [25]:

# Define a function to categorize ages
def categorize_age(age):
    if age <= 365:  # 0-1 years
        return '0-1 years'
    elif age <= 3 * 365:  # 1-3 years
        return '1-3 years'
    else:
        return 'Over 3 years'


In [35]:
#df['filed_date'] = pd.to_datetime(df['filed_date'], format='%Y-%m-%d')
processed_df['pending_age'] = (processed_df['end_date'] - processed_df['filed_date']).dt.days
processed_df = processed_df[processed_df['pending_age'] >= 0]


In [ ]:
processed_df['age_group'] = processed_df['pending_age'].apply(categorize_age)
backlog_category = processed_df.pivot_table(index='court', columns='age_group', values='unique_number', aggfunc='count', fill_value=0)


# Save the data to csv

In [249]:
column_order = ['murder', 'criminal_appeal', 'criminal_application', 
                'revision', 'suit', 'civil_appeal', 'misc_application', 
                'constitutional_petition', 'judicial_review',
                  'bankruptcy_and_insolvency', 'tax_appeal', 
                  'adoption', 'divorce', 'probate_and_admin']
# Total missing per month
missing_per_month.to_csv(f'{output_path}/missing_outcomes_per_month.csv', index=False)

# Total filed cases
filed_cases.to_csv(f'{output_path}/filed_cases.csv', columns=column_order, index=True)

# Total concluded cases
concluded_cases.to_csv(f'{output_path}/concluded_cases.csv', columns=column_order, index=True)

# Total filed, concluded and CCR per month
monthly_cases.to_csv(f'{output_path}/monthly_cases.csv', index=False)

# Productivity per court
df_pivot.to_csv(f'{output_path}/productivity.csv', index=True)

# Average time to conclude
average_time_to_conclude.to_csv(f'{output_path}/average_time_to_conclude.csv', index=True)

# Time lines
proportion_resolved_within_timeline_per_court.to_csv(f'{output_path}/proportion_resolved_within_timeline.csv', index=True)

judge_pivot.to_csv(f'{output_path}/judge_productivity.csv', index=True)

judge_court_productivity.to_csv(f'{output_path}/judge_court_productivity.csv', index=False)

court_pivot.to_csv(f'{output_path}/court_productivity.csv', index=True)

matters_handled.to_csv(f'{output_path}/judge_matters_handled.csv', index=False)

adjourned_per_court.to_csv(f'{output_path}/adjourned_per_court.csv', index=False)

adjourn_proportion.to_csv(f'{output_path}/adjourn_proportion.csv', index=False)

pending_cases.to_csv(f'{output_path}/hc_pending_cases.csv', index=True)

backlog_category.to_csv(f'{output_path}/hc_backlog.csv', index=True)


# raw data of pending cases(next period baseline)
processed_df.to_csv(f'{output_path}/raw_pending_cases.csv', index=False)


In [84]:
proportion_resolved_within_timeline_per_court.to_csv(f'{output_path}/proportion_resolved_within_timeline.csv', index=True)

In [92]:
### COA Saving
output_path = '/home/arch/devel/data/Report'
'''
column_order = ['murder', 'criminal_appeal', 'criminal_application', 
                'revision', 'suit', 'civil_appeal', 'misc_application', 
                'constitutional_petition', 'judicial_review',
                  'bankruptcy_and_insolvency', 'tax_appeal', 
                  'adoption', 'divorce', 'probate_and_admin']

'''
column_order  = ['COA Criminal Appeal', 'Criminal Applications', 'Civil Appeal', 'Civil Applications']
# Total missing per month
missing_per_month.to_csv(f'{output_path}/missing_outcomes_per_month.csv', index=False)

# Total filed cases
filed_cases.to_csv(f'{output_path}/filed_cases.csv', index=True)

# Total concluded cases
concluded_cases.to_csv(f'{output_path}/concluded_cases.csv', index=True)

# Total filed, concluded and CCR per month
monthly_cases.to_csv(f'{output_path}/monthly_cases.csv', index=False)

# Productivity per court
df_pivot.to_csv(f'{output_path}/productivity.csv', index=True)

# Average time to conclude
average_time_to_conclude.to_csv(f'{output_path}/average_time_to_conclude.csv', index=True)

# Time lines
proportion_resolved_within_timeline_per_court.to_csv(f'{output_path}/proportion_resolved_within_timeline.csv', index=True)


court_pivot.to_csv(f'{output_path}/court_productivity.csv', index=True)


adjourned_per_court.to_csv(f'{output_path}/adjourned_per_court.csv', index=False)

adjourn_proportion.to_csv(f'{output_path}/adjourn_proportion.csv', index=False)


In [137]:
proportion_resolved_within_timeline_per_court.to_csv(f'{output_path}/proportion_resolved_within_timeline.csv', index=True)

In [138]:
df.to_csv(f'{output_path}/coa_cases.csv', index=False)

### Exploratory analysis

In [177]:
def get_cases_per_quarter(df, column):
    # Group by quarters and count cases
    cases_per_quarter = df.groupby(pd.Grouper(key='activity_date', freq='QE'))[column].sum()

    # Reset index to make the quarters a column
    cases_per_quarter = cases_per_quarter.reset_index()

    # Rename the columns
    cases_per_quarter.columns = ['quarter', f'cases_{column}']

    return cases_per_quarter

In [105]:
def get_case_nature_per_quarter(df: pd.DataFrame, column: str, nature: str):
   
    # Set up date range for quarters
    quarters = df['activity_date'].dt.to_period('Q')

    # Filter cases where concluded == 1 and case nature == 'civil'
    filtered_df = df[(df[column] == 1) & (df['nature'] == nature)]

    # Group by quarter and count cases
    cases_per_quarter = filtered_df.groupby(quarters).size()

    return cases_per_quarter

In [7]:
# drop if activity_date is null
df = df.dropna(subset=['activity_date'])

In [ ]:
quarterly_concluded =  get_cases_per_quarter(df, 'concluded')

In [204]:
quarterly_adjourned =  get_cases_per_quarter(df, 'adjourned')
quarterly_adjournable =  get_cases_per_quarter(df, 'adjournable')
quarterly_concluded =  get_cases_per_quarter(df, 'concluded')
quarterly_registered =  get_cases_per_quarter(df, 'registered')
# merge quarterly_concluded and quarterly_registered on quarter column
merged_quarterly = pd.merge(quarterly_adjourned, quarterly_adjourned, quarterly_concluded, quarterly_registered, on='quarter')

In [207]:
# merge quarterly_concluded and quarterly_registered on quarter column
merged_quarterly = pd.merge(quarterly_adjourned, quarterly_adjourned, quarterly_concluded, quarterly_registered, on='quarter')

In [211]:
df.drop('court', inplace=True, axis=1)

In [ ]:
df.groupby(['court', 'month'])['concluded'].sum().reset_index(name='count')

In [106]:
quarterly_concluded_civil = get_case_nature_per_quarter(df, 'concluded', 'Civil')
quarterly_registered_civil = get_case_nature_per_quarter(df, 'registered', 'Civil')

quarterly_concluded_criminal = get_case_nature_per_quarter(df, 'concluded', 'Criminal')
quarterly_registered_criminal = get_case_nature_per_quarter(df, 'registered', 'Criminal')

In [ ]:
# merge quarterly_concluded_civil quarterly_registered_civil quarterly_concluded_criminal quarterly_registered_criminal on quarter column 
merged_civil_criminal = pd.merge(quarterly_concluded_civil, quarterly_registered_civil, on='quarter')
merged_civil_criminal = pd.merge(merged_civil_criminal, quarterly_concluded_criminal, on='quarter')
merged_civil_criminal = pd.merge(merged_civil_criminal, quarterly_registered_criminal, on='quarter')

In [114]:
merged_civil_criminal.to_csv(f'{output_path}/quarterly_case_nature.csv', index=True)

TODO
### adjourn by event
### adjourn by case type

### backlog by case type
### check if there a courts that resolved more cases than pending

In [ ]:
#df[(df['court'] == 'Milimani Commercial and Tax') & (df['activity_date_year'] == 2024)].groupby('judge_1').size().reset_index(name='count')
# group by case_type if outcome == 'Ruling Delivered- Case Closed' 
#df[df['productivity'] == 'judgment'].groupby('case_category').size().reset_index(name='count')
#court_productivity = df[df['productivity'] == 'ruling']
#court_productivity_pivot = court_productivity.pivot_table(index='court', columns='case_category', values='concluded', aggfunc='sum', fill_value=0)
#productivity_pivot = court_productivity_pivot.rename_axis(columns=None)
#df[df['productivity'] == 'ruling'].groupby('case_category').size().reset_index(name='count')
#df[df['outcome'] == 'Ruling Delivered- Accused Discharged'].groupby(['court', 'case_category'])['court'].size().reset_index(name='count')


In [44]:
pending_count = pending_df.groupby(['court', 'case_category']).size().reset_index(name='pending_count')

In [46]:
pending_pivot = pending_count.pivot_table(index='court', columns='case_category', values='pending_count', fill_value=0)

In [67]:
pending_pivot.to_csv('/home/arch/devel/data/pending_case_types.csv', index=True)

In [18]:
def categorize_concluded_cases(df):
    """
    Categorize concluded cases into '1-3 year' and 'over 3 year' based on age column.

    Args:
        df (pandas.DataFrame): DataFrame containing the data.

    Returns:
        pandas.DataFrame: DataFrame with an additional column 'age_category'.
    """
    # Filter only concluded cases
    df_concluded = df[df['concluded'] == 1]

    # Calculate age in years
    df_concluded['age_years'] = df_concluded['age'] / 365

    # Categorize based on age
    df_concluded.loc[:, 'age_category'] = df_concluded['age_years'].apply(lambda x: '1-3 year' if 1 <= x <= 3 else 'over 3 year')

    # Merge back to original DataFrame
    df = pd.merge(df, df_concluded[['case_number', 'age_category']], on='case_number', how='left')

    return df


In [ ]:
df = categorize_concluded_cases(df)

In [384]:
df.groupby('court').size().reset_index(name='count')

,court,count
0,Bomet High Court_High Court Div,4076
1,Bungoma High Court_High Court Div,11632
2,Busia High Court_High Court Civil,2987
3,Busia High Court_High Court Criminal,1555
4,Chuka High Court_High Court Div,3057
...,...,...
71,Nyeri High Court_High Court Family,4007
72,Siaya High Court_High Court Div,4601
73,Thika High Court_High Court Div,6357
74,Vihiga High Court_High Court Div,2854


In [382]:
# groupby judge_1 if court == 'Milimani Commercial and Tax' 
#df(df['court'] == 'Milimani Commercial and Tax').groupby([ 'judge_1',])['outcome'].size().reset_index(name='count')
df[df['court'] == 'Meru'].groupby(['judge_1'])['outcome'].size().reset_index(name='count')

,judge_1,count


In [ ]:
# group by court if case_type == 'Murder Case' and registered == 1
df[(df['case_type'] == 'Criminal Appeal') & (df['outcome'] == 'File Transferred')].groupby('court').size().reset_index(name='count')

In [ ]:
['comingfor', 'outcome',  'male_applicant', 'female_applicant',
       'organization_applicant', 'male_defendant', 'female_defendant',
       'organization_defendant', 'legalrep', 'court',
       'activity_date', 'filed_date', 'activity_date_year',
       'activity_date_month', 'nature', 'case_category', 'case_number',
       'registered', 'concluded', 'productivity', 'age', 'time_lines',
       'adjourned', 'adjournable']

In [184]:
   # Drop duplicates based on 'number_on_file' while keeping the first occurrence
df_unique_cases = df.drop_duplicates(subset='case_number', keep='last')
    

In [190]:
df_unique_cases.shape

(103156, 53)

In [ ]:
filed_df = df[df['registered'] == 1]
male_applicants = filed_df[(filed_df['male_applicant'] == 1) & (filed_df['case_category'] != 'murder')].groupby('case_category').size().reset_index(name='count')
female_applicants = filed_df[(filed_df['female_applicant'] == 1) & (filed_df['case_category'] != 'murder')].groupby('case_category').size().reset_index(name='count')
male_accused = filed_df[(filed_df['male_defendant'] == 1) & (filed_df['case_category'] == 'murder')].groupby('case_category').size().reset_index(name='count')
male_murder_defendants = filed_df[(filed_df['male_defendant'] == 1) & (filed_df['case_category'] == 'murder')].groupby('case_category').size().reset_index(name='count')
female_murder_defendants = filed_df[(filed_df['female_defendant'] == 1) & (filed_df['case_category'] == 'murder')].groupby('case_category').size().reset_index(name='count')
applicants = pd.merge(female_applicants, male_applicants, on='case_category', how='outer', suffixes=('_female', '_male'))
applicants.to_csv(f'{output_path}/applicants_gender.csv', index=False)

